# ಕಾರ್ಟ್‌ಪೋಲ್ ಸಮತೋಲನ ಮಾಡಲು RL ತರಬೇತಿ

ಈ ನೋಟ್ಬುಕ್ [AI for Beginners Curriculum](http://aka.ms/ai-beginners) ಭಾಗವಾಗಿದೆ. ಇದಕ್ಕೆ ಪ್ರೇರಣೆ ನೀಡಿದ್ದು [ಅಧಿಕೃತ PyTorch ಟ್ಯುಟೋರಿಯಲ್](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) ಮತ್ತು [ಈ Cartpole PyTorch ಅನುಷ್ಠಾನ](https://github.com/yc930401/Actor-Critic-pytorch).

ಈ ಉದಾಹರಣೆಯಲ್ಲಿ, ನಾವು RL ಬಳಸಿ ಕಾರ್ಟ್ ಮೇಲೆ ಸಮತೋಲನ ಸಾಧಿಸಲು ಮಾದರಿಯನ್ನು ತರಬೇತಿಮಾಡುತ್ತೇವೆ, ಕಾರ್ಟ್ ಎಡಕ್ಕೆ ಮತ್ತು ಬಲಕ್ಕೆ ಹೋರಿಜಾಂಟಲ್ ಮಾಪಕದಲ್ಲಿ ಚಲಿಸಬಹುದು. ನಾವು [OpenAI Gym](https://www.gymlibrary.ml/) ಪರಿಸರವನ್ನು ಪೋಲ್ ಅನುಕರಿಸಲು ಬಳಸುತ್ತೇವೆ.

> **Note**: ನೀವು ಈ ಪಾಠದ ಕೋಡ್ ಅನ್ನು ಸ್ಥಳೀಯವಾಗಿ (ಉದಾ. Visual Studio Code ನಿಂದ) ಚಾಲನೆ ಮಾಡಬಹುದು, ಆ ಸಂದರ್ಭದಲ್ಲಿ ಅನುಕರಣ ಹೊಸ ವಿಂಡೋದಲ್ಲಿ ತೆರೆಯುತ್ತದೆ. ಆನ್‌ಲೈನ್‌ನಲ್ಲಿ ಕೋಡ್ ಚಾಲನೆ ಮಾಡುವಾಗ, ಕೆಲವು ಬದಲಾವಣೆಗಳನ್ನು ಮಾಡಬೇಕಾಗಬಹುದು, ಇದನ್ನು [ಇಲ್ಲಿ](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7) ವಿವರಿಸಲಾಗಿದೆ.

ನಾವು ಮೊದಲು Gym ಸ್ಥಾಪಿತವಾಗಿದೆ ಎಂದು ಖಚಿತಪಡಿಸಿಕೊಳ್ಳೋಣ:


In [ ]:
import sys
!{sys.executable} -m pip install gym

ಈಗ ನಾವು CartPole ಪರಿಸರವನ್ನು ರಚಿಸಿ ಅದನ್ನು ಹೇಗೆ ಕಾರ್ಯಗತಗೊಳಿಸಬಹುದು ಎಂದು ನೋಡೋಣ. ಒಂದು ಪರಿಸರದಲ್ಲಿ ಕೆಳಗಿನ ಗುಣಲಕ್ಷಣಗಳಿರುತ್ತವೆ:

* **ಕ್ರಿಯೆ ಸ್ಥಳ** ಎಂದರೆ ಸಿಮ್ಯುಲೇಶನ್‌ನ ಪ್ರತಿ ಹಂತದಲ್ಲಿ ನಾವು ನಡೆಸಬಹುದಾದ ಸಾಧ್ಯವಾದ ಕ್ರಿಯೆಗಳ ಸಮೂಹ
* **ನಿರೀಕ್ಷಣಾ ಸ್ಥಳ** ಎಂದರೆ ನಾವು ಮಾಡಬಹುದಾದ ನಿರೀಕ್ಷಣೆಗಳ ಸ್ಥಳ


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

ನಾವು ಸಿಮ್ಯುಲೇಶನ್ ಹೇಗೆ ಕಾರ್ಯನಿರ್ವಹಿಸುತ್ತದೆ ಎಂದು ನೋಡೋಣ. ಕೆಳಗಿನ ಲೂಪ್ ಸಿಮ್ಯುಲೇಶನ್ ಅನ್ನು ಚಲಾಯಿಸುತ್ತದೆ, `env.step` ಮುಗింపు ಧ್ವಜ `done` ಅನ್ನು ಹಿಂತಿರುಗಿಸುವವರೆಗೆ. ನಾವು ಕ್ರಿಯೆಗಳನ್ನು ಯಾದೃಚ್ಛಿಕವಾಗಿ ಆಯ್ಕೆಮಾಡುತ್ತೇವೆ `env.action_space.sample()` ಬಳಸಿ, ಅಂದರೆ ಪ್ರಯೋಗ ಬಹುಶಃ ಬಹಳ ಬೇಗ ವಿಫಲವಾಗಬಹುದು (CartPole ಪರಿಸರವು CartPole ನ ವೇಗ, ಅದರ ಸ್ಥಾನ ಅಥವಾ ಕೋನವು ನಿರ್ದಿಷ್ಟ ಮಿತಿಗಳನ್ನು ಮೀರಿ ಹೋದಾಗ ಮುಗಿಯುತ್ತದೆ).

> ಸಿಮ್ಯುಲೇಶನ್ ಹೊಸ ವಿಂಡೋದಲ್ಲಿ ತೆರೆಯುತ್ತದೆ. ನೀವು ಕೋಡ್ ಅನ್ನು ಹಲವಾರು ಬಾರಿ ಚಲಾಯಿಸಿ ಅದರ ವರ್ತನೆ ಹೇಗಿದೆ ಎಂದು ನೋಡಬಹುದು.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

ನೀವು ಗಮನಿಸಬಹುದು, ಅವಲೋಕನಗಳಲ್ಲಿ 4 ಸಂಖ್ಯೆಗಳು ಇರುತ್ತವೆ. ಅವುಗಳೇ:
- ಕಾರ್ಟ್‌ನ ಸ್ಥಾನ
- ಕಾರ್ಟ್‌ನ ವೇಗ
- ಕಂಬದ ಕೋನ
- ಕಂಬದ ತಿರುಗುವ ದರ

`rew` ಎಂದರೆ ನಾವು ಪ್ರತಿ ಹಂತದಲ್ಲಿ ಪಡೆಯುವ ಬಹುಮಾನ. CartPole ಪರಿಸರದಲ್ಲಿ ಪ್ರತಿ ಸಿಮ್ಯುಲೇಶನ್ ಹಂತಕ್ಕೆ ನೀವು 1 ಪಾಯಿಂಟ್ ಬಹುಮಾನ ಪಡೆಯುತ್ತೀರಿ, ಮತ್ತು ಗುರಿ ಒಟ್ಟು ಬಹುಮಾನವನ್ನು ಗರಿಷ್ಠಗೊಳಿಸುವುದು, ಅಂದರೆ CartPole ಬಿದ್ದಿಲ್ಲದೆ ಸಮತೋಲನ ಸಾಧಿಸುವ ಸಮಯವನ್ನು ಹೆಚ್ಚಿಸುವುದು.

ಪುನರಾವರ್ತಿತ ಕಲಿಕೆಯಲ್ಲಿ, ನಮ್ಮ ಗುರಿ ಒಂದು **ನೀತಿ** $\pi$ ಅನ್ನು ತರಬೇತುಗೊಳಿಸುವುದು, ಅದು ಪ್ರತಿ ಸ್ಥಿತಿಗೆ $s$ ನಾವು ಯಾವ ಕ್ರಿಯೆ $a$ ತೆಗೆದುಕೊಳ್ಳಬೇಕು ಎಂದು ಹೇಳುತ್ತದೆ, ಅಂದರೆ $a = \pi(s)$.

ನೀವು ಪ್ರಾಬಬಿಲಿಸ್ಟಿಕ್ ಪರಿಹಾರವನ್ನು ಬಯಸಿದರೆ, ನೀತಿ ಪ್ರತಿ ಕ್ರಿಯೆಗೆ ಸಾಧ್ಯತೆಗಳ ಸಮೂಹವನ್ನು ನೀಡುತ್ತದೆ ಎಂದು ಭಾವಿಸಬಹುದು, ಅಂದರೆ $\pi(a|s)$ ಎಂದರೆ ಸ್ಥಿತಿ $s$ ನಲ್ಲಿ ನಾವು ಕ್ರಿಯೆ $a$ ತೆಗೆದುಕೊಳ್ಳಬೇಕಾದ ಸಾಧ್ಯತೆ.

## ನೀತಿ ಗ್ರೇಡಿಯಂಟ್ ವಿಧಾನ

ಸರಳವಾದ RL ಅಲ್ಗೋರಿದಮ್, **ನೀತಿ ಗ್ರೇಡಿಯಂಟ್** ಎಂದು ಕರೆಯಲ್ಪಡುವುದರಲ್ಲಿ, ನಾವು ಮುಂದಿನ ಕ್ರಿಯೆಯನ್ನು ಊಹಿಸಲು ನ್ಯೂರಲ್ ನೆಟ್‌ವರ್ಕ್ ಅನ್ನು ತರಬೇತುಗೊಳಿಸುತ್ತೇವೆ.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

ನಾವು ಹಲವಾರು ಪ್ರಯೋಗಗಳನ್ನು ನಡೆಸಿ ನೆಟ್‌ವರ್ಕ್ ಅನ್ನು ತರಬೇತುಗೊಳಿಸುವೆವು, ಮತ್ತು ಪ್ರತಿ ರನ್ ನಂತರ ನಮ್ಮ ನೆಟ್‌ವರ್ಕ್ ಅನ್ನು ನವೀಕರಿಸುವೆವು. ಪ್ರಯೋಗವನ್ನು ನಡೆಸಿ ಫಲಿತಾಂಶಗಳನ್ನು (ಹೆಸರಿಸಿದಂತೆ **ಟ್ರೇಸ್**) ಹಿಂತಿರುಗಿಸುವ ಫಂಕ್ಷನ್ ಅನ್ನು ವ್ಯಾಖ್ಯಾನಿಸೋಣ - ಎಲ್ಲಾ ಸ್ಥಿತಿಗಳು, ಕ್ರಿಯೆಗಳು (ಮತ್ತು ಅವುಗಳ ಶಿಫಾರಸು ಮಾಡಿದ ಸಾಧ್ಯತೆಗಳು), ಮತ್ತು ಬಹುಮಾನಗಳು:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

ನೀವು ಒಂದು ಎಪಿಸೋಡ್ ಅನ್ನು ತರಬೇತಿಗೊಳ್ಳದ ನೆಟ್‌ವರ್ಕ್‌ನೊಂದಿಗೆ ನಡೆಸಬಹುದು ಮತ್ತು ಒಟ್ಟು ಬಹುಮಾನ (ಅಥವಾ ಎಪಿಸೋಡ್‌ನ ಉದ್ದ) ತುಂಬಾ ಕಡಿಮೆ ಎಂದು ಗಮನಿಸಬಹುದು:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

ನೀತಿ ಗ್ರೇಡಿಯಂಟ್ ಆಲ್ಗೋರಿದಮ್‌ನ ಒಂದು ಕಷ್ಟಕರ ಅಂಶವೆಂದರೆ **ರಿಯಾಯಿತಿಯ наградыಗಳನ್ನು** ಬಳಸುವುದು. ಆ ಕಲ್ಪನೆ ಎಂದರೆ ನಾವು ಆಟದ ಪ್ರತಿ ಹಂತದಲ್ಲಿ ಒಟ್ಟು ರಿಯಾಯಿತಿಗಳ ವೆಕ್ಟರ್ ಅನ್ನು ಲೆಕ್ಕಹಾಕುತ್ತೇವೆ, ಮತ್ತು ಈ ಪ್ರಕ್ರಿಯೆಯಲ್ಲಿ ನಾವು ಆರಂಭಿಕ ರಿಯಾಯಿತಿಗಳನ್ನು ಕೆಲವು ಗುಣಾಂಕ $gamma$ ಬಳಸಿ ರಿಯಾಯಿತಿಗೊಳಿಸುತ್ತೇವೆ. ನಾವು ಫಲಿತಾಂಶ ವೆಕ್ಟರ್ ಅನ್ನು ಸಾಮಾನ್ಯೀಕರಿಸುತ್ತೇವೆ, ಏಕೆಂದರೆ ನಾವು ಅದನ್ನು ತರಬೇತಿಗೆ ಪ್ರಭಾವ ಬೀರುವ ತೂಕವಾಗಿ ಬಳಸುತ್ತೇವೆ:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

ಈಗ ನಿಜವಾದ ತರಬೇತಿಯನ್ನು ಮಾಡೋಣ! ನಾವು 300 ಎಪಿಸೋಡ್‌ಗಳನ್ನು ನಡೆಸುತ್ತೇವೆ, ಮತ್ತು ಪ್ರತಿ ಎಪಿಸೋಡ್‌ನಲ್ಲಿ ನಾವು ಕೆಳಗಿನವುಗಳನ್ನು ಮಾಡುತ್ತೇವೆ:

1. ಪ್ರಯೋಗವನ್ನು ನಡೆಸಿ ಮತ್ತು ಟ್ರೇಸ್ ಅನ್ನು ಸಂಗ್ರಹಿಸಿ  
1. ತೆಗೆದುಕೊಂಡ ಕ್ರಿಯೆಗಳು ಮತ್ತು ಭವಿಷ್ಯವಾಣಿ ಪ್ರಾಬಬಿಲಿಟಿಗಳ ನಡುವಿನ ವ್ಯತ್ಯಾಸವನ್ನು (`gradients`) ಲೆಕ್ಕಹಾಕಿ. ವ್ಯತ್ಯಾಸ ಕಡಿಮೆ ಇದ್ದರೆ, ನಾವು ಸರಿಯಾದ ಕ್ರಿಯೆಯನ್ನು ತೆಗೆದುಕೊಂಡಿದ್ದೇವೆ ಎಂದು ಹೆಚ್ಚು ಖಚಿತವಾಗಿರುತ್ತೇವೆ.  
1. ಡಿಸ್ಕೌಂಟೆಡ್ ರಿವಾರ್ಡ್‌ಗಳನ್ನು ಲೆಕ್ಕಿಸಿ ಮತ್ತು gradients ಅನ್ನು ಡಿಸ್ಕೌಂಟೆಡ್ ರಿವಾರ್ಡ್‌ಗಳೊಂದಿಗೆ ಗುಣಿಸಿ - ಇದರಿಂದ ಹೆಚ್ಚಿನ ರಿವಾರ್ಡ್‌ಗಳಿರುವ ಹಂತಗಳು ಕಡಿಮೆ ರಿವಾರ್ಡ್‌ಗಳಿಗಿಂತ ಅಂತಿಮ ಫಲಿತಾಂಶದ ಮೇಲೆ ಹೆಚ್ಚು ಪ್ರಭಾವ ಬೀರುತ್ತವೆ.  
1. ನಮ್ಮ ನ್ಯೂರಲ್ ನೆಟ್‌ವರ್ಕ್‌ಗೆ ನಿರೀಕ್ಷಿತ ಗುರಿ ಕ್ರಿಯೆಗಳು ಭಾಗಶಃ ರನ್ ಸಮಯದಲ್ಲಿ ಭವಿಷ್ಯವಾಣಿ ಪ್ರಾಬಬಿಲಿಟಿಗಳಿಂದ ಮತ್ತು ಭಾಗಶಃ ಲೆಕ್ಕಹಾಕಿದ gradients ನಿಂದ ತೆಗೆದುಕೊಳ್ಳಲಾಗುತ್ತದೆ. gradients ಮತ್ತು ರಿವಾರ್ಡ್‌ಗಳನ್ನು ಎಷ್ಟು ಪ್ರಮಾಣದಲ್ಲಿ ಪರಿಗಣಿಸಬೇಕೆಂದು ನಿರ್ಧರಿಸಲು ನಾವು `alpha` ಪರಿಮಾಣವನ್ನು ಬಳಸುತ್ತೇವೆ - ಇದನ್ನು ರೀಇನ್ಫೋರ್ಸ್ಮೆಂಟ್ ಆಲ್ಗೋರಿದಮ್‌ನ *ಲರ್ನಿಂಗ್ ರೇಟ್* ಎಂದು ಕರೆಯುತ್ತಾರೆ.  
1. ಕೊನೆಗೆ, ನಾವು ನಮ್ಮ ನೆಟ್‌ವರ್ಕ್ ಅನ್ನು ಸ್ಥಿತಿಗಳ ಮತ್ತು ನಿರೀಕ್ಷಿತ ಕ್ರಿಯೆಗಳ ಮೇಲೆ ತರಬೇತಿಗೊಳಿಸಿ, ಮತ್ತು ಈ ಪ್ರಕ್ರಿಯೆಯನ್ನು ಪುನರಾವರ್ತಿಸುತ್ತೇವೆ.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

ಈಗ ಫಲಿತಾಂಶವನ್ನು ನೋಡಲು ರೆಂಡರಿಂಗ್ ಜೊತೆಗೆ ಎಪಿಸೋಡ್ ಅನ್ನು ಚಾಲನೆ ಮಾಡೋಣ:


In [ ]:
_ = run_episode(render=True)

ನೀವು ಈಗ ಧ್ರುವವನ್ನು ಚೆನ್ನಾಗಿ ಸಮತೋಲನಗೊಳಿಸಬಹುದೆಂದು ಕಾಣಬಹುದು!

## ನಟ-ವಿಮರ್ಶಕ ಮಾದರಿ

ನಟ-ವಿಮರ್ಶಕ ಮಾದರಿ ನೀತಿ ಗ್ರೇಡಿಯಂಟ್‌ಗಳ ಮುಂದಿನ ಅಭಿವೃದ್ಧಿಯಾಗಿದೆ, ಇದರಲ್ಲಿ ನಾವು ನೀತಿ ಮತ್ತು ಅಂದಾಜು ಮಾಡಿದ ಬಹುಮಾನಗಳನ್ನು ಎರಡನ್ನೂ ಕಲಿಯಲು ನ್ಯೂರಲ್ ನೆಟ್‌ವರ್ಕ್ ನಿರ್ಮಿಸುತ್ತೇವೆ. ನೆಟ್‌ವರ್ಕ್‌ಗೆ ಎರಡು ಔಟ್‌ಪುಟ್‌ಗಳು ಇರುತ್ತವೆ (ಅಥವಾ ನೀವು ಇದನ್ನು ಎರಡು ಪ್ರತ್ಯೇಕ ನೆಟ್‌ವರ್ಕ್‌ಗಳಾಗಿ ನೋಡಬಹುದು):
* **ನಟ** ನೀತಿ ಗ್ರೇಡಿಯಂಟ್ ಮಾದರಿಯಂತೆ ನಮಗೆ ರಾಜ್ಯದ ಪ್ರಾಬಬಿಲಿಟಿ ವಿತರಣೆಯನ್ನು ನೀಡುವ ಮೂಲಕ ತೆಗೆದುಕೊಳ್ಳಬೇಕಾದ ಕ್ರಿಯೆಯನ್ನು ಶಿಫಾರಸು ಮಾಡುತ್ತದೆ
* **ವಿಮರ್ಶಕ** ಆ ಕ್ರಿಯೆಗಳಿಂದ ಬಹುಮಾನ ಏನು ಆಗಬಹುದು ಎಂದು ಅಂದಾಜು ಮಾಡುತ್ತದೆ. ಅದು ನೀಡಲಾದ ರಾಜ್ಯದಲ್ಲಿ ಭವಿಷ್ಯದಲ್ಲಿ ಒಟ್ಟು ಅಂದಾಜು ಮಾಡಿದ ಬಹುಮಾನಗಳನ್ನು ಹಿಂತಿರುಗಿಸುತ್ತದೆ.

ಹೀಗೆ ಒಂದು ಮಾದರಿಯನ್ನು ವ್ಯಾಖ್ಯಾನಿಸೋಣ:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

ನಾವು ನಮ್ಮ `discounted_rewards` ಮತ್ತು `run_episode` ಫಂಕ್ಷನ್‌ಗಳನ್ನು ಸ್ವಲ್ಪ ಬದಲಾಯಿಸಬೇಕಾಗುತ್ತದೆ:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


ಈಗ ನಾವು ಮುಖ್ಯ ತರಬೇತಿ ಲೂಪ್ ಅನ್ನು ನಡೆಸಲಿದ್ದೇವೆ. ಸರಿಯಾದ ನಷ್ಟ ಕಾರ್ಯಗಳನ್ನು ಲೆಕ್ಕಹಾಕಿ ಮತ್ತು ನೆಟ್‌ವರ್ಕ್ ಪರಿಮಾಣಗಳನ್ನು ನವೀಕರಿಸುವ ಮೂಲಕ ಕೈಯಿಂದ ನೆಟ್‌ವರ್ಕ್ ತರಬೇತಿ ಪ್ರಕ್ರಿಯೆಯನ್ನು ಬಳಸಲಿದ್ದೇವೆ:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## ಸಾರಾಂಶ

ಈ ಡೆಮೋದಲ್ಲಿ ನಾವು ಎರಡು RL ಅಲ್ಗೋರಿದಮ್‌ಗಳನ್ನು ನೋಡಿದ್ದೇವೆ: ಸರಳ ಪಾಲಿಸಿ ಗ್ರೇಡಿಯಂಟ್ ಮತ್ತು ಹೆಚ್ಚು ಸುಧಾರಿತ ಆಕ್ಟರ್-ಕ್ರಿಟಿಕ್. ಆ ಅಲ್ಗೋರಿದಮ್‌ಗಳು ಸ್ಥಿತಿ, ಕ್ರಿಯೆ ಮತ್ತು ಬಹುಮಾನ ಎಂಬ抽象概念ಗಳೊಂದಿಗೆ ಕಾರ್ಯನಿರ್ವಹಿಸುತ್ತವೆ ಎಂದು ನೀವು ನೋಡಬಹುದು - ಆದ್ದರಿಂದ ಅವುಗಳನ್ನು ಬಹು ವಿಭಿನ್ನ ಪರಿಸರಗಳಲ್ಲಿ ಅನ್ವಯಿಸಬಹುದು.

ರೀಇನ್ಫೋರ್ಸ್ಮೆಂಟ್ ಲರ್ನಿಂಗ್ ನಮಗೆ ಸಮಸ್ಯೆಯನ್ನು ಪರಿಹರಿಸಲು ಅತ್ಯುತ್ತಮ ತಂತ್ರವನ್ನು ಕೇವಲ ಅಂತಿಮ ಬಹುಮಾನವನ್ನು ನೋಡಿ ಕಲಿಯಲು ಅವಕಾಶ ನೀಡುತ್ತದೆ. ನಮಗೆ ಲೇಬಲ್ ಮಾಡಲಾದ ಡೇಟಾಸೆಟ್‌ಗಳ ಅಗತ್ಯವಿಲ್ಲ ಎಂಬುದು ನಮ್ಮ ಮಾದರಿಗಳನ್ನು ಉತ್ತಮಗೊಳಿಸಲು ಅನೇಕ ಬಾರಿ ಸಿಮ್ಯುಲೇಶನ್‌ಗಳನ್ನು ಪುನರಾವರ್ತಿಸಲು ಅವಕಾಶ ನೀಡುತ್ತದೆ. ಆದಾಗ್ಯೂ, RL ನಲ್ಲಿ ಇನ್ನೂ ಅನೇಕ ಸವಾಲುಗಳಿವೆ, ಮತ್ತು ನೀವು AI ಯ ಈ ಆಸಕ್ತಿದಾಯಕ ಕ್ಷೇತ್ರದಲ್ಲಿ ಹೆಚ್ಚು ಗಮನಹರಿಸಲು ನಿರ್ಧರಿಸಿದರೆ ಅವುಗಳನ್ನು ಕಲಿಯಬಹುದು.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸ್ವೀಕಾರ**:  
ಈ ದಸ್ತಾವೇಜು [Co-op Translator](https://github.com/Azure/co-op-translator) ಎಂಬ AI ಅನುವಾದ ಸೇವೆಯನ್ನು ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ಶುದ್ಧತೆಯತ್ತ ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ತಪ್ಪುಗಳು ಅಥವಾ ಅಸತ್ಯತೆಗಳು ಇರಬಹುದು ಎಂದು ದಯವಿಟ್ಟು ಗಮನಿಸಿ. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಸ್ತಾವೇಜನ್ನು ಅಧಿಕೃತ ಮೂಲವೆಂದು ಪರಿಗಣಿಸಬೇಕು. ಮಹತ್ವದ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪು ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವಿಕೆ ಅಥವಾ ತಪ್ಪು ವಿವರಣೆಗಳಿಗೆ ನಾವು ಹೊಣೆಗಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
